In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-14 12:39:49.615271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 12:39:49.771085: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 12:39:50.314828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-14 12:39:50.314896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
sys.path.append(str(Path("../../ssl").resolve()))

In [4]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.pseudo_label.pseudo_label import PseudoLabelTrainer
from src.trainers.pseudo_label.pseudo_label_config import PseudoLabelTrainerConfig
from src.data_loaders.pseudo_label.pseudo_label import PseudoLabelDataLoader
from src.data_loaders.pseudo_label.pseudo_label import PseudoLabelDataLoaderConfig

# Description

In this notebook, the a model will be trained on the CIFAR-10 dataset using only 20% of the labelled dataset. Unlabelled data will be used in training.

## Set up Experiment

In [5]:
class TrainerConfig(PseudoLabelTrainerConfig):
    num_epochs = 200
    t1 = 100
    t2 = 150
    alpha = 3.0

train_config = TrainerConfig()

In [6]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [7]:
class DataLoaderConfig(PseudoLabelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

data_loader_config = DataLoaderConfig()

## Get Datasets

In [8]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [9]:
# get train dataset by subsampling 20% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [10]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-14 12:39:52.248950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 12:39:52.254847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 12:39:52.255327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 12:39:52.256592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [12]:
# create train dataset
train_data = PseudoLabelDataLoader(train_data, data_loader_config)(training = True)

In [13]:
# create test dataset
val_data = PseudoLabelDataLoader(val_data, data_loader_config)(training = False)

In [14]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 782
Validation dataset size: 157


# Train Model

In [15]:
model = PiModel(model_config)()

In [16]:
trainer = PseudoLabelTrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [17]:
trainer.train()

2022-11-14 12:39:54.235053: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.08. Validation loss is : 1.88. Validation acc. is : 26.87.
Training loss at epoch 1 is : 1.78. Validation loss is : 1.80. Validation acc. is : 30.52.
Training loss at epoch 2 is : 1.64. Validation loss is : 1.72. Validation acc. is : 36.73.
Training loss at epoch 3 is : 1.54. Validation loss is : 1.53. Validation acc. is : 43.41.
Training loss at epoch 4 is : 1.46. Validation loss is : 1.41. Validation acc. is : 48.17.
Training loss at epoch 5 is : 1.39. Validation loss is : 1.35. Validation acc. is : 49.90.
Training loss at epoch 6 is : 1.30. Validation loss is : 1.30. Validation acc. is : 52.92.
Training loss at epoch 7 is : 1.23. Validation loss is : 1.27. Validation acc. is : 55.58.
Training loss at epoch 8 is : 1.17. Validation loss is : 1.25. Validation acc. is : 55.35.
Training loss at epoch 9 is : 1.11. Validation loss is : 1.22. Validation acc. is : 55.94.
Training loss at epoch 10 is : 1.08. Validation loss is : 1.16. Validation acc. is : 58.16